In [1]:
from pyspark.sql import SparkSession

input_data = r"F:\DataSamples\DataExplorer\Census\Census_Surnames"
output_wh = r"F:\Spark_Data_Test\WH_Names"

spark = SparkSession.builder \
    .appName("Laptop Census") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .enableHiveSupport() \
    .config("spark.sql.warehouse.dir", output_wh) \
    .getOrCreate()

In [2]:
df_base = spark.read.csv(input_data + r"\*.csv", header=True, inferSchema=True)
df_base.createOrReplaceTempView("census_surnames")
df_base.count()

162254

In [26]:
from pyspark.sql.functions import *
df_base.printSchema()
df_base.withColumn("first_letter", substring("name", 1,1)).orderBy(desc("count")).show(10, truncate=False)
# df_base.select(left("name", 1)).orderBy(desc("count")).show(10, truncate=False)


root
 |-- name: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- prop100k: double (nullable = true)
 |-- cum_prop100k: double (nullable = true)
 |-- pctwhite: string (nullable = true)
 |-- pctblack: string (nullable = true)
 |-- pctapi: string (nullable = true)
 |-- pctaian: string (nullable = true)
 |-- pct2prace: string (nullable = true)
 |-- pcthispanic: string (nullable = true)

+---------------+----+--------+--------+------------+--------+--------+------+-------+---------+-----------+------------+
|name           |rank|count   |prop100k|cum_prop100k|pctwhite|pctblack|pctapi|pctaian|pct2prace|pcthispanic|first_letter|
+---------------+----+--------+--------+------------+--------+--------+------+-------+---------+-----------+------------+
|ALL OTHER NAMES|0   |29312001|9936.97 |9936.97     |66.65   |8.53    |7.97  |0.86   |2.32     |13.67      |A           |
|SMITH          |1   |2442977 |828.19  |828.19      |70.9    |23.11   

In [11]:
df_base.select("name").distinct().count()
# df_base.agg(countDistinct(col("name"))).show()

162254

In [38]:
from pyspark.sql.window import Window

next_record = Window.orderBy("cum_prop100k")
next_alpha = Window.partitionBy(substring("name", 1,1)).orderBy("cum_prop100k")
next_alpha_run = Window.partitionBy(substring("name", 1,1)).orderBy("cum_prop100k").rangeBetween(Window.unboundedPreceding, Window.currentRow)
alpha_run = Window.partitionBy(substring("name", 1,1)).orderBy("cum_prop100k").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

df_range = df_base.filter(col("name") != "ALL OTHER NAMES").withColumn("lower_bound", lag(col("cum_prop100k"), 1, 0).over(next_record)) \
    .withColumn("next_alpha_name", lead(col("name"), 1).over(next_alpha)) \
    .withColumn("letter_sum", sum(col("count")).over(next_alpha)) \
    .withColumn("letter_running_total", sum(col("count")).over(next_alpha_run)) \
    .withColumn("letter_tot", sum(col("count")).over(alpha_run)) \
    .withColumn("letter_running_freq", sum(col("count")).over(next_alpha_run) / sum(col("count")).over(alpha_run)) \
    .withColumn("avg_1", avg(col("count")).over(next_alpha)) \
    .withColumn("avg_2", avg(col("count")).over(next_alpha_run)) \
    .withColumn("first_letter", substring("name", 1, 1)) \
    .withColumn("last_letter", substring("name", -1, 1))

df_range.orderBy("cum_prop100k").show(10, truncate=False)

+---------+----+-------+--------+------------+--------+--------+------+-------+---------+-----------+-----------+---------------+----------+--------------------+----------+-------------------+---------+---------+------------+-----------+
|name     |rank|count  |prop100k|cum_prop100k|pctwhite|pctblack|pctapi|pctaian|pct2prace|pcthispanic|lower_bound|next_alpha_name|letter_sum|letter_running_total|letter_tot|letter_running_freq|avg_1    |avg_2    |first_letter|last_letter|
+---------+----+-------+--------+------------+--------+--------+------+-------+---------+-----------+-----------+---------------+----------+--------------------+----------+-------------------+---------+---------+------------+-----------+
|SMITH    |1   |2442977|828.19  |828.19      |70.9    |23.11   |0.5   |0.89   |2.19     |2.4        |0.0        |SANCHEZ        |2442977   |2442977             |25056728  |0.09749784568839155|2442977.0|2442977.0|S           |H          |
|JOHNSON  |2   |1932812|655.24  |1483.42     |58

In [39]:
df_range.filter(col("first_letter") == "A").orderBy("cum_prop100k").show(10, truncate=False)


+---------+----+------+--------+------------+--------+--------+------+-------+---------+-----------+-----------+---------------+----------+--------------------+----------+-------------------+------------------+------------------+------------+-----------+
|name     |rank|count |prop100k|cum_prop100k|pctwhite|pctblack|pctapi|pctaian|pct2prace|pcthispanic|lower_bound|next_alpha_name|letter_sum|letter_running_total|letter_tot|letter_running_freq|avg_1             |avg_2             |first_letter|last_letter|
+---------+----+------+--------+------------+--------+--------+------+-------+---------+-----------+-----------+---------------+----------+--------------------+----------+-------------------+------------------+------------------+------------+-----------+
|ANDERSON |15  |784404|265.92  |6375.92     |75.17   |18.93   |0.61  |0.74   |2.11     |2.44       |6110.0     |ALLEN          |784404    |784404              |10080086  |0.07781719322632763|784404.0          |784404.0          |A     

In [ ]:
from pyspark.sql.types import *

df_range.groupBy("first_letter").agg(
    count("*").alias("letter_cnt"),
    sum(col("count")).alias("letter_sum"),
    sum(round(col("cum_prop100k"), 2)).alias("letter_cum_prop100k"),
    avg(when(((col("pctaian") == '(S)') | col("pctaian").isNull()), 0) \
        .otherwise(col("pctaian").cast(DoubleType()))).alias("avg_pctaian"),
).show(20, truncate=False)

+------------+----------+----------+--------------------+-------------------+
|first_letter|letter_cnt|letter_sum|letter_cum_prop100k |avg_pctaian        |
+------------+----------+----------+--------------------+-------------------+
|K           |9246      |8669408   |7.780379563500003E8 |0.41398550724637684|
|F           |5722      |8981809   |4.7415834726000005E8|0.48603809856693475|
|Q           |437       |642349    |3.617222983999999E7 |0.6448283752860411 |
|E           |3338      |4932313   |2.765225303E8       |0.6270970641102457 |
|T           |6406      |9413187   |5.3508646858000004E8|0.9182875429285047 |
|B           |14204     |22587641  |1.1754116904900014E9|0.7607040270346377 |
|Y           |1100      |1684921   |9.185053141000001E7 |1.6518090909090908 |
|M           |13521     |25610240  |1.11770524926E9     |0.4157333037497226 |
|L           |8354      |13045257  |6.92154241440001E8  |0.7524299736653103 |
|V           |4257      |4657504   |3.575125713999999E8 |0.28505